# Import packages

In [ ]:
from packaging import version

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
#import seaborn as sns
from scipy import stats
import tensorboard as tb

mpl.rcParams['axes.labelpad'] = 4.0

# Experiment results

In [ ]:
df = pd.read_csv("实验数据csv.csv",index_col=[0,1], header=[0,1])
df

# Poison v Clean

In [ ]:
poisoned_model = df.xs("poisoned", level=1)
clean_model = df.xs("clean", level=1)

BM_trigger = df.xs("poisoned", level=1).loc[:,(slice(None), 'w/ trigger pattern')]

BM_clean = df.xs("poisoned", level=1).loc[:,(slice(None), 'w/o trigger pattern')]

CM_clean = df.xs("clean", level=1).loc[:,(slice(None), 'w/o trigger pattern')]

#fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(8, 12))
vec_dataset_mean = BM_clean.mean(axis=0)
vec_dataset_std = BM_clean.std(axis=0)


average_clean = BM_clean.mean(axis=1)
BM_clean["mean"] = average_clean
BM_clean

average_trigger = BM_trigger.mean(axis=1)
BM_trigger["mean"] = average_trigger
vec_trigger_model = BM_trigger.loc[:,"mean"]

average = CM_clean.mean(axis=1)
CM_clean["mean"] = average
vec_clean_model = CM_clean.loc[:,"mean"]

dataset_0_pct = pd.Series([10, 1, 0.5, 0.475, 10], dtype=float, index=vec_dataset_mean.index)


dataset_clean_compare = pd.DataFrame({
    "mean" : vec_dataset_mean,
    "std" : vec_dataset_std,
    "true" : dataset_0_pct
})

dataset_clean_compare = dataset_clean_compare.reset_index(level=1,drop=True)
dataset_clean_compare = dataset_clean_compare.reset_index(col_level=1)


#dataset_clean_compare.columns = ["Dataset","Accuracy","pct. of class 0" ]
dataset_clean_compare.to_latex("BM_clean.tex",index=False, float_format=f'%.{2}f')

# Poison rate selection

In [ ]:
rate = pd.DataFrame({
    "rate" : pd.Series([0.95,0.8,0.65,0.5,0.35,0.2,0.05]),
    "acc" : pd.Series([98.0,98.100,97.98,97.74,97.4,96.86,92.46])
    
})
ax = rate.plot(x="rate",y="acc",ylabel="backdoored model accuracy", xlabel="poison rate",legend=False, fontsize=14)
ax.set_xlabel('poison rate',fontsize=14.0)
ax.set_ylabel('backdoored model accuracy',fontsize=14.0)

plt.savefig("rate.pdf")

# Functionality-preserving of inverse backdoored models

In [ ]:
# Create sample Series
series1 = vec_trigger_model
series2 = vec_clean_model

# Set the categories (variables) for the radar plot
categories = series1.index

# Create a figure and axis for the radar plot
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': 'polar'})

# Convert values to radians
theta = np.linspace(0, 2*np.pi, len(categories) + 1, endpoint=True)

# Plot data for Series 1
values1 = series1.values
values1_closed = np.concatenate([values1, [values1[0]]])
ax.plot(theta, values1_closed, label='backdoored models')
ax.fill(theta, values1_closed, alpha=0.2)

# Plot data for Series 2
values2 = series2.values
values2_closed = np.concatenate([values2, [values2[0]]])
ax.plot(theta, values2_closed, label='clean models')
ax.fill(theta, values2_closed, alpha=0.2)

# Set the labels for each category
ax.set_xticks(theta[:-1], categories,)
ax.set_xticklabels(categories,fontsize=14.0)

label_text = []
for i, label in enumerate(ax.get_xticklabels()):
    label.set_y(-0.0)
    if i == 0:
        label.set_y(-0.05)
    elif i==1:
        label.set_y(-0.05)
    elif i==2:
        label.set_y(-0.05)
    elif i == 3:
        label.set_y(-0.25)
    elif i == 4:
        label.set_y(-0.1)
    elif i == 5:
        label.set_y(-0.1)
        

# Set the radial grid
ax.set_rgrids([30, 60, 90], labels=['30', '60', '90'], angle=45)

# Add legend
ax.legend(loc='upper right',bbox_to_anchor=(1.5,1.0),fontsize=14.0)
ax.spines["bottom"] = ax.spines["inner"]
# Add a title
# plt.title('Comparison between inverse backdoored and clean model, accuracy averaged over five datasets')
# Show the radar plo

plt.savefig('radar.pdf',bbox_inches='tight')

# 32 by 32 trigger with random patter

In [ ]:
key_rs = pd.read_csv("实验数据key.csv",index_col=[0])
key_rs

In [ ]:
#key_rs.columns = ['img_size@224', 'img_size@288', 'baseline']
print(key_rs[1:])
ax = key_rs[1:].plot()
plt.axhline(y=97.86, color='red', linestyle='--', label='Horizontal Line at y=0.5')
ax.set_ylabel('accuracy')
ax.set_xlabel('pct. of correct key pattern')